In [3]:
from openai import OpenAI
from pydantic import BaseModel
from typing import Optional
import json
from os import environ

In [29]:
import openai
import os
openai.api_key = os.getenv('OPENAI_API_KEY')
openai.base_url = os.getenv('OPENAI_BASE_URL')
openai.api_type = 'openai'

try:
    response = openai.chat.completions.create(
        model = "gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."}, 
            {"role": "user", "content": "When did Messi last score?"} 
        ]
    )
    print("API Key is valid!")
    print(response)
except Exception as e:
    print(f"An error occurred: {e}")



An error occurred: Connection error.


In [8]:
import openai
import os

client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY'),
    base_url=os.getenv('OPENAI_BASE_URL', 'https://api.openai.com')
)


print(client)
response = client.chat.completions.create(

    model= os.getenv('OPENAI_MODEL_DEPLOYMENT'), # model to send to the proxy

    messages = [

        {

            "role": "user",

            "content": "this is a test request, write a short poem"

        }

    ]

)


/tmp/ipykernel_1261/3228221808.py:22: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chat([message])


APIConnectionError: Connection error.

In [4]:
# Step 1. Instantiating your TavilyClient
from tavily import TavilyClient
tavily = TavilyClient(api_key=environ['TAVILY_API_KEY'])

# Step 2. Executing a simple search query
response = tavily.search("When did Messi last score?")

# Step 3. That's it! You've done a Tavily Search!
print(response)

{'query': 'When did Messi last score?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Lionel Messi Stats, Goals, Records, Assists, Cups and more - FBref.com', 'url': 'https://fbref.com/en/players/d70ce98e/Lionel-Messi', 'content': 'Match Logs (Miscellaneous Stats)\nGoal Logs by Competition\nScouting Report\nInter Miami(9-7-18, 27th place in Major League Soccer)\nRoster\nShow entire roster\nArgentina\nRoster\nShow entire roster\nOn this page:\nLionel Messi Scouting Report\nLionel Messi Scouting Report\nSimilar Players to Lionel Messi\nSimilar Players to Lionel Messi\nLast 5 Matches\nMore:\nMatch Logs\n•\nGoal Logs\nPlayer News\nStandard Stats: Domestic Leagues\nShooting: Domestic Leagues\nPassing: Domestic Leagues\nPass Types: Domestic Leagues\nGoal and Shot Creation: Domestic Leagues\nDefensive Actions: Domestic Leagues\nPossession: Domestic Leagues\nPlaying Time: Instagram:\n@leomessi\nMLS\n6\n373\n1\n2\n2.6\n2.6\n1.4\n27\n3\n* see our coverage note

In [6]:
def internet_search(query):
    """
    This functions searches the internet for the provided query string. Always use me to answer any question from the user
    """
    response = tavily.search(query)
    return response

In [7]:
import inspect

def function_to_schema(func) -> dict:
    type_map = {
        str: "string",
        int: "integer",
        float: "number",
        bool: "boolean",
        list: "array",
        dict: "object",
        type(None): "null",
    }

    try:
        signature = inspect.signature(func)
    except ValueError as e:
        raise ValueError(
            f"Failed to get signature for function {func.__name__}: {str(e)}"
        )

    parameters = {}
    for param in signature.parameters.values():
        try:
            param_type = type_map.get(param.annotation, "string")
        except KeyError as e:
            raise KeyError(
                f"Unknown type annotation {param.annotation} for parameter {param.name}: {str(e)}"
            )
        parameters[param.name] = {"type": param_type}

    required = [
        param.name
        for param in signature.parameters.values()
        if param.default == inspect._empty
    ]

    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": (func.__doc__ or "").strip(),
            "parameters": {
                "type": "object",
                "properties": parameters,
                "required": required,
            },
        },
    }

In [8]:
messages = []

tools = [internet_search]
tool_schemas = [function_to_schema(tool) for tool in tools]

In [9]:
response = client.chat.completions.create(
            model="openai.gpt-4o",
            messages=[{"role": "user", "content": "when did messi last score a goal?"}],
            tools=tool_schemas,
        )
message = response.choices[0].message

message.tool_calls[0].function

Function(arguments='{"query":"Lionel Messi last goal date"}', name='internet_search')

In [10]:
tools = [internet_search]


def run_full_turn(system_message, tools, messages):

    num_init_messages = len(messages)
    messages = messages.copy()

    while True:

        # turn python functions into tools and save a reverse map
        tool_schemas = [function_to_schema(tool) for tool in tools]
        tools_map = {tool.__name__: tool for tool in tools}

        # === 1. get openai completion ===
        response = client.chat.completions.create(
            model="openai.gpt-4o",
            messages=[{"role": "system", "content": system_message}] + messages,
            tools=tool_schemas or None,
        )
        message = response.choices[0].message
        messages.append(message)

        if message.content:  # print assistant response
            print("Assistant:", message.content)

        if not message.tool_calls:  # if finished handling tool calls, break
            break

        # === 2. handle tool calls ===

        for tool_call in message.tool_calls:
            result = execute_tool_call(tool_call, tools_map)

            result_message = {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(result),
            }
            print("tool result:", result)
            messages.append(result_message)

    # ==== 3. return new messages =====
    return messages[num_init_messages:]


def execute_tool_call(tool_call, tools_map):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    print(f"Assistant: {name}({args})")

    # call corresponding function with provided arguments
    return tools_map[name](**args)


messages = []
while True:
    user = input("User: ")
    if user == "exit":
        break
    messages.append({"role": "user", "content": user})

    new_messages = run_full_turn("you are a helpful assistant", tools, messages)
    messages.extend(new_messages)

    

Assistant: Hello! How can I assist you today?
Assistant: It seems like your message was empty. How can I help you today?
Assistant: internet_search({'query': 'current wind speed in Ithaca, NY'})
tool result: {'query': 'current wind speed in Ithaca, NY', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in Ithaca, NY Metro Area, New York, USA - timeanddate.com', 'url': 'https://www.timeanddate.com/weather/@12213491', 'content': 'Current weather in Ithaca, NY Metro Area and forecast for today, tomorrow, and next 14 days. Sign in. News. News Home; Astronomy News; Time Zone News; ... Wind Speed: 5 mph: 1 mph: 5 mph: 11 mph: 8 mph: 10 mph: 14 mph: Wind Direction: NW', 'score': 0.99415743, 'raw_content': None}, {'title': 'Wind Forecast: wind speed & gusts - Windy.app', 'url': 'https://windy.app/forecast2/spot/603970/Ithaca+ny', 'content': 'Wind direction is Southeast, wind speed varies between 4.5 and 13.4 mph with gusts up to 31.2 mph. The sky is clou